In [19]:
import pandas as pd
import numpy as np

import pymysql
from google.oauth2 import service_account
from google.cloud import bigquery
from pandas_gbq import to_gbq

import os
from dotenv import load_dotenv


In [20]:
load_dotenv()

mysql_con = pymysql.connect(
    host=os.getenv("host"),
    port=int(os.getenv("port")),
    user=os.getenv("user"),
    password=os.getenv("password"),
    db=os.getenv("db")
)
cursor = mysql_con.cursor()

## ETL

In [21]:
query1 = ("""select * from olist_orders_dataset""")
query2 = ("""select * from olist_order_reviews_dataset""")
query3 = ("""select * from olist_order_payments_dataset""")
query4 = ("""select * from olist_order_items_dataset""")
query5 = ("""select * from product_category_name_translation""")
query6 = ("""select * from olist_products_dataset""")
query7 = ("""select * from olist_geolocation_dataset""")
query8 = ("""select * from olist_sellers_dataset""")
query9 = ("""select * from olist_customers_dataset""")
query10 = ("""select * from olist_closed_deals_dataset""")
query11 = ("""select * from olist_marketing_qualified_leads_dataset""")

orders_df = pd.read_sql(query1, mysql_con)
order_review_df = pd.read_sql(query2, mysql_con)
order_payment_df = pd.read_sql(query3, mysql_con)
order_items_df = pd.read_sql(query4, mysql_con)
product_category_name_df = pd.read_sql(query5, mysql_con)
products_df = pd.read_sql(query6, mysql_con)
geo_df = pd.read_sql(query7, mysql_con)
sellers_df = pd.read_sql(query8, mysql_con)
customers_df = pd.read_sql(query9, mysql_con)
closed_deals_df = pd.read_sql(query10, mysql_con)
mql_df = pd.read_sql(query11, mysql_con)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28168\324209136.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders_df = pd.read_sql(query1, mysql_con)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28168\324209136.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_review_df = pd.read_sql(query2, mysql_con)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_28168\324209136.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_payment_df = pd.read_sql(query3, mysql_con)
C:\Users\Pub

In [22]:
print(orders_df.isnull().sum())
print()
print(order_review_df.isnull().sum())
print()
print(order_payment_df.isnull().sum())
print()
print(order_items_df.isnull().sum())
print()
print(product_category_name_df.isnull().sum())
print()
print(products_df.isnull().sum())
print()
print(geo_df.isnull().sum())
print()
print(sellers_df.isnull().sum())
print()
print(customers_df.isnull().sum())
print()
print(closed_deals_df.isnull().sum())
print()
print(mql_df.isnull().sum())
print()


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

product_category_name            0
product_category_name_english    0
dtype: int64

product_id          

In [ ]:
orders_df = orders_df[['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp']].drop_duplicates()

order_items_df = order_items_df[['order_id', 'order_item_id', 'product_id', 'price', 'freight_value']].drop_duplicates()
order_items_df['revenue'] = order_items_df['order_item_id']*(order_items_df['price']+order_items_df['freight_value'])
order_items_df.rename(columns={'order_item_id':'count'}, inplace=True)

products_df = products_df[['product_id', 'product_category_name']]
join_product_df= pd.merge(products_df,product_category_name_df, how='inner', on='product_category_name')
join_product_df = join_product_df[['product_id', 'product_category_name_english']].rename(columns={'product_category_name_english':'product_category_name'})

order_payment_df=order_payment_df[['order_id', 'payment_type']]

order_review_df = order_review_df[['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date']]

customers_df = customers_df[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix']]

geo_df = geo_df[['geolocation_zip_code_prefix','geolocation_lat', 'geolocation_lng', 'geolocation_city']]

sellers_df = sellers_df[['seller_id', 'seller_zip_code_prefix']]
order_review_df = order_review_df.dropna(subset=['review_comment_message'])
order_review_df['review_comment_title'] = order_review_df['review_comment_title'].fillna('.')
order_review_df = order_review_df[['review_id', 'order_id', 'review_score', 'review_creation_date']]

mql_df.origin = mql_df.origin.str.replace('_', ' ')
mql_df = mql_df[(mql_df['origin'].notnull()) & (mql_df['origin'] != '')]


##### Bigquery에 삽입

In [12]:
SERVICE_ACCOUNT_FILE = "./bigquery_key/api_key.json"  # 키 json 파일
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
project_id = "driven-net-452904-q3"
dataset_name = "ecommerce_project"
client = bigquery.Client(credentials=credentials, project=project_id)

def import_bigquery_data(query):
    query_job = client.query(query)
    return query_job.to_dataframe()

In [13]:
to_gbq(orders_df, f"{dataset_name}.orders", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.orders")
to_gbq(order_items_df, f"{dataset_name}.order_items", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.order_items")
to_gbq(join_product_df, f"{dataset_name}.products", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.products")
to_gbq(order_payment_df, f"{dataset_name}.payments", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.payments")
to_gbq(order_review_df, f"{dataset_name}.reviews", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.reviews")
to_gbq(customers_df, f"{dataset_name}.customers", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.customers")
to_gbq(sellers_df, f"{dataset_name}.sellers", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.sellers")
to_gbq(geo_df, f"{dataset_name}.geolocation", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.geolocation")
to_gbq(closed_deals_df, f"{dataset_name}.closed_deals", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.closed_deals")
to_gbq(mql_df, f"{dataset_name}.mql", project_id=project_id, if_exists="replace")
print(f"✅ BigQuery에 데이터 적재 완료: {project_id}.{dataset_name}.mql")



100%|██████████| 1/1 [00:00<00:00, 999.12it/s]


✅ BigQuery에 데이터 적재 완료: driven-net-452904-q3.ecommerce_project.orders


100%|██████████| 1/1 [00:00<?, ?it/s]


✅ BigQuery에 데이터 적재 완료: driven-net-452904-q3.ecommerce_project.order_items


100%|██████████| 1/1 [00:00<?, ?it/s]


...

✅ BigQuery에 데이터 적재 완료: driven-net-452904-q3.ecommerce_project.geolocation


100%|██████████| 1/1 [00:00<?, ?it/s]


✅ BigQuery에 데이터 적재 완료: driven-net-452904-q3.ecommerce_project.closed_deals


100%|██████████| 1/1 [00:00<?, ?it/s]

✅ BigQuery에 데이터 적재 완료: driven-net-452904-q3.ecommerce_project.mql


In [14]:
mysql_con.close()